In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
X, y = make_regression()

In [ ]:
model = Ridge()
alphas = np.logspace(-5, 5, 10000)

In [ ]:
cv = RandomizedSearchCV(model, param_distributions={"alpha": alphas}, cv=3, n_iter=100, scoring='neg_mean_squared_error')

In [ ]:
cv.fit(X, y)

In [ ]:
import polars as pl

df = pl.DataFrame(cv.cv_results_)

In [ ]:
import altair as alt

# Make the x-axis logarithmic
alt.Chart(df).mark_line().encode(x=alt.X("param_alpha").scale(type="log"), y="mean_test_score")

## Enter Optuna

The above works, but lets now consider a slightly different system: Optuna!

In [ ]:
! uv pip install optuna

In [ ]:
import optuna

def objective(trial: optuna.Trial):
    x = trial.suggest_uniform('alpha', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study(direction='minimize')

In [ ]:
study.optimize(objective, n_trials=100)

In [ ]:
study.best_trial

In [ ]:
study.best_params

In [ ]:
study.trials_dataframe()

Let's now do this but for scikit-learn. 

In [ ]:
from sklearn.model_selection import cross_val_score

def objective(trial: optuna.Trial):
    alpha = trial.suggest_float('alpha', 1e-5, 1e5, log=True)
    model = Ridge(alpha=alpha)
    return cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error').mean()

study = optuna.create_study(storage="sqlite:///ridge-demo.db", direction= 'maximize')
study.optimize(objective, n_trials=100)

In [ ]:
trials_df = study.trials_dataframe()

# Make the x-axis logarithmic
alt.Chart(trials_df[['value', "params_alpha"]]).mark_line().encode(x=alt.X("params_alpha").scale(type="log"), y="value")

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

There is some more fancy stuff that it can do though. 

In [ ]:
from sklearn.model_selection import cross_val_score
from optuna.artifacts import upload_artifact, download_artifact, FileSystemArtifactStore
from pathlib import Path
from skops.io import dump
from uuid import uuid4

artifacts_path = Path("./artifacts")
artifacts_path.mkdir(exist_ok=True)
artifact_store = FileSystemArtifactStore(base_path=artifacts_path)

def objective(trial: optuna.Trial):
    alpha = trial.suggest_float('alpha', 1e-5, 1e5, log=True)
    model = Ridge(alfpha=alpha)
    score = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error').mean()
    model.fit(X, y)
    file_path = f"model_{trial.number}.skops"
    dump(model, file_path)
    artifact_id = upload_artifact(artifact_store=artifact_store, file_path=file_path, study_or_trial=trial)
    trial.set_user_attr("artifact_id", artifact_id)
    return score

study = optuna.create_study(storage="sqlite:///ridge2.db", direction= 'maximize')
study.optimize(objective, n_trials=100)

In [ ]:
study.trials[0]

In [ ]:
best_artifact_id = study.best_trial.user_attrs.get("artifact_id")
download_artifact(artifact_store=artifact_store, artifact_id=best_artifact_id, file_path="best_model.skops")

In [ ]:
from skops.io import load

load("best_model.skops")

But lets now see if we can influence the search, because we have not been doing any of that yet. 

Lets start by talking about pruning. 

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

import optuna

X, y = load_iris(return_X_y=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
classes = np.unique(y)


def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    clf = SGDClassifier(alpha=alpha)
    n_train_iter = 100

    for step in range(n_train_iter):
        clf.partial_fit(X_train, y_train, classes=classes)

        intermediate_value = clf.score(X_valid, y_valid)
        trial.report(intermediate_value, step)

        if trial.should_prune():
            raise optuna.TrialPruned()

    return clf.score(X_valid, y_valid)


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=30, interval_steps=10
    ),
)
study.optimize(objective, n_trials=20)

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

import optuna

X, y = load_iris(return_X_y=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
classes = np.unique(y)


def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    clf = SGDClassifier(alpha=alpha)
    n_train_iter = 100

    for step in range(n_train_iter):
        clf.partial_fit(X_train, y_train, classes=classes)

        intermediate_value = clf.score(X_valid, y_valid)
        trial.report(intermediate_value, step)

        if trial.should_prune():
            raise optuna.TrialPruned()

    return clf.score(X_valid, y_valid)


study = optuna.create_study(
    direction="maximize",
)
study.optimize(objective, n_trials=20)

But we can also adapt the search here.

In [ ]:
import optuna
from optuna.samplers import TPESampler, RandomSampler


def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)
    z = trial.suggest_float("z", -10, 10)
    return x**2 + y**2 + z**2

n_trials = 500

study_tpe = optuna.create_study(sampler=TPESampler(n_startup_trials=50))
study_tpe.optimize(objective, n_trials=n_trials)

In [ ]:
import pandas as pd

pltr = pd.concat([
    study_norm.trials_dataframe().assign(kind="norm"),
    study_tpe.trials_dataframe().assign(kind="tpe"),
])[["number", "value", "kind"]]


alt.Chart(pltr).mark_line().encode(x="number", y="value", color="kind").interactive().properties(width=750)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study_norm)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study_tpe)

In [ ]:
a